# Processing data with pandas II

Finnish university students are encouraged to use the CSC Notebooks platform.<br/>
<a href="https://notebooks.csc.fi/"><img alt="CSC badge" src="https://img.shields.io/badge/launch-CSC%20notebook-blue.svg" style="vertical-align:text-bottom"></a>

**Note**: We do not recommended using Binder for this lesson.

This week we will continue developing our skills using [pandas](https://pandas.pydata.org/) to process real data. 

## Motivation

![Finland April 2019](img/Finland-April-2019.png)
*Source: [https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa)*

April 2019 was the [second warmest April on record globally](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa), and the warmest on record at 13 weather stations in Finland. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data, and investigate the claim that April 2019 was the warmest on record across Finland.

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files

## Input data

In the lesson this week we are using weather observation data from Finland [downloaded from NOAA](https://www.ncei.noaa.gov/cdo-web/). You will be working with data from a total of 15 different weather observation stations in Finland.

### Downloading the data

**Note**: The instructions about downloading the data only apply to students working with Jupyter Lab installed on their own computers. If you are using the CSC Notebooks you do not need follow these steps!

If you are working on a Jupyter environment installed on your own computer (i.e., not the CSC notebooks), the first step for today's lesson is to get the data. We recommend using the command line tool [wget](https://www.gnu.org/software/wget/) for downloading the data. wget is hopefully already installed on your computer.

First, you need to open a new terminal window in Jupyter Lab (from **File** -> **New** -> **Terminal**). Once the terminal window is open, you will need to navigate to the L6 directory:

```bash
cd path/to/L6/
```

where `path/to/` should be replaced with the correct path for the Lesson 6 materials on your computer. Once in the correct directory, you can confirm this by typing:

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-5-assertions.ipynb
```

Next, you can download the data files using wget:

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-full.tar.gz
```

After the download completes, you can extract the data files using the tar command:

```bash
tar zxvf Finland-weather-data-full.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by listing the contents of the data-folder:

```bash
ls data
```

You should see something like the following:

```bash
028360.txt           029070.txt           029440.txt           029740.txt  6367598020644inv.txt
028690.txt           029110.txt           029500.txt           029810.txt  6367598020644stn.txt
028750.txt           029170.txt           029700.txt           029820.txt
028970.txt           029350.txt           029720.txt           3505doc.txt
```

Now you should be all set to proceed with the lesson!

## About the data

As part of the download there are a number of files that describe the weather data. These *metadata* files include:

- A list of stations: [6367598020644stn.txt](metadata/6367598020644stn.txt)
- Details about weather observations at each station: [6367598020644inv.txt](metadata/6367598020644inv.txt)
- A data description (i.e., column names): [3505doc.txt](metadata/3505doc.txt)

The input data for this week are separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

We will develop our analysis workflow using data for a single station. Then, we will repeat the same process for all the stations.

## Reading the data

In order to get started, let's first import pandas: 

At this point, we can already have a quick look at the data file `029440.txt` for Tampere Pirkkala and how it is structured. We can notice at least two things we need to consider when reading in the data:

#### Input data structure

- **Delimiter:** The data are **separated with a varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can use either `sep='\s+'` or `delim_whitespace=True` (but not both at the same time). In this case, we prefer to use `delim_whitespace` parameter.

- **No Data values:** No data values in the NOAA data are coded with varying number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.

In [ ]:
# Define absolute path to the file
fp = r"/home/jovyan/shared/data/L6/029440.txt"

In [ ]:
# Read data using varying amount of spaces as separator and specifying * characters as NoData values
# And don't worry about the warning message generated when running this cell
data = pd.read_csv(
    fp, delim_whitespace=True, na_values=["*", "**", "***", "****", "*****", "******"]
)

**Note**: If you are working with Jupyter Lab installed locally on your own computer and extracted your files in the L6 notebooks directory, then you can use the file path below.

```python
# Define relative path to the file
fp = r"data/029440.txt"
```

Let's see how the data looks by printing the first five rows with the `head()` function:

All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

A description for all these columns is available in the metadata file [data/3505doc.txt](metadata/3505doc.txt). 

### Reading in the data once again

This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [ ]:
# Read in only selected columns
data = pd.read_csv(
    fp,
    delim_whitespace=True,
    usecols=["USAF", "YR--MODAHRMN", "DIR", "SPD", "GUS", "TEMP", "MAX", "MIN"],
    na_values=["*", "**", "***", "****", "*****", "******"],
)

# Check the dataframe
data.head()

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

As we saw above some of the column names are a bit awkward and difficult to interpret. Luckily, it is easy to alter labels in a pandas DataFrame using the [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) function. In order to change the column names, we need to tell pandas how we want to rename the columns using a dictionary that lists old and new column names

Let's first check again the current column names in our DataFrame:

We can define the new column names using a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) where we list `key: value` pairs, in which the original column name (the one which will be replaced) is the key and the new column name is the value.

#### Dictionaries

A [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a specific type of data structure in Python for storing key-value pairs. In this course, we will use dictionaries mainly when renaming columns in a pandas DataFrame, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).

Let's change the following:

- `YR--MODAHRMN` to `TIME`
- `SPD` to `SPEED`
- `GUS` to `GUST`

In [ ]:
# Create the dictionary with old and new names


# Let's see what the variable new_names look like


In [ ]:
# Check the data type of the new_names variable


From above we can see that we have successfully created a new dictionary. 

Now we can change the column names by passing that dictionary using the parameter `columns` in the `rename()` function:

In [ ]:
# Rename the columns


# Print the new columns
print(data.columns)

Perfect, now our column names are easier to understand and use. 

### Check your understanding

The temperature values in our data files are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, let's rename the column `TEMP` to `TEMP_F`. Let's also rename `USAF` to `STATION_NUMBER`.

In [ ]:
# Create the dictionary with old and new names


# Rename the columns


# Check the output
data.head()

## Data properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with the data analysis. Let's check the:

- Number of rows and columns

- Top and bottom rows

- Data types of the columns

- Descriptive statistics

Here we can see that there are varying number of observations per column (look at the `count` row above), because some of the columns have missing values.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply our own functions to data in a pandas DataFrame.

First, we will define a function for the temperature conversion, and then we will apply this function for each Fahrenheit value on each row of the DataFrame. The output celsius values will be stored in a new column called `TEMP_C`.

To begin we will see how we can apply the function row-by-row using a `for` loop and then we will learn how to apply the function to all rows more efficiently all at once.

### Defining the function

For both of these approaches, we first need to define our function to convert temperature from Fahrenheit to Celsius.

In [ ]:
def fahr_to_celsius(temp_fahrenheit):
    """Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)

    Returns
    -------

    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8

    return converted_temp

To make sure everything is working properly, let's test the function with a known value.

Let's also print out the first rows of our data frame to see our input data before further processing.

### Iterating over rows

We can use the function one row at a time using a `for` loop and the [iterrows()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) method. This will allow us to iterate row by row using `iterrows()` in a `for` loop to repeat a given process *for each row in a pandas DataFrame*. Please note that iterating over rows is a rather inefficient approach, but it is still useful to understand the logic behind the iteration.

When using the `iterrows()` method it is important to understand that `iterrows()` accesses not only the values of one row, but also the `index` of the row as well. 

Let's start with a simple for loop that goes through each row in our DataFrame.

**Note**: We use single quotes to select the column `TEMP_F` of the row in the example below. This is because using double quotes would result in a `SyntaxError` since Python would interpret this as the end of the string for the `print()` function.

In [ ]:
# Iterate over the rows
for idx, row in data.iterrows():

    # Print the index value
    print(f"Index: {idx}")

    # Print the row
    print(f"Temp F: {row['TEMP_F']}\n")

    break

#### Breaking a loop

When developing code in a `for` loop, you do not always need to go through the entire loop in order to test things out. 
The [break](https://docs.python.org/3/reference/simple_stmts.html#break) statement in Python terminates the current loop whereever it is placed and we can use it here just to check out the values on the first row (based on the first iteration in the `for` loop.
This can be helpful when working with a large data file or dataset, because you might not want to print thousands of values to the screen!
For more information, check out [this tutorial](https://www.tutorialspoint.com/python/python_break_statement.htm).

We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values in that column using the `fahr_to_celsius` function we defined earlier.

In [ ]:
# Create an empty float column for the output values
data["TEMP_C"] = 0.0

# Iterate over the rows
for idx, row in data.iterrows():

    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row["TEMP_F"])

    # Update the value of 'Celsius' column with the converted value
    data.at[idx, "TEMP_C"] = celsius

#### Reminder: .at or .loc?

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html) is faster than [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html), which is designed for accessing groups of rows and columns. 

Finally, let's see how our DataFrame looks like now after the calculations above.

### Applying the function

pandas DataFrames and Series have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parentheses!) as an argument to the `apply()` method. Let's start by applying the function to the `TEMP_F` column that contains the temperature values in Fahrenheit.

The results look logical, so we can store them permanently into the `TEMP_C` column (overwriting the old values).

We can also apply the function on several columns at once. Furthermore, we can re-order the dataframe at the same time.

### Check your understanding

Convert `'TEMP_F'`, `'MIN'`, `'MAX'` to Celsius by applying the function like we did above and store the outputs to  new columns `'TEMP_C'`, `'MIN_C'`, `'MAX_C'`.

In [ ]:
# Enter your solution below


**Note**: Applying the function on all columns using `data.apply(fahr_to_celsius)` would not give an error for the data we are using, but the results also do not make much sense for columns where input data are something other than Fahrenheit temperatures.

You might also notice that our conversion function would also allow us to 
pass one column or the entire dataframe as a parameter. For example, like this: `fahr_to_celsius(data["TEMP_F"])`. However, the code is perhaps easier to follow when using the apply method.

Let's check now take a look at the DataFrame contents.

#### Should I use .iterrows() or .apply()?

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping through DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 

At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!

## Parsing dates

As part of this lesson, we eventually want to group our data based on month in order to see whether the temperatures in April of 2019 were warmer than average. Currently, the date and time information is stored in the column `TIME` (which was originally titled `YR--MODAHRMN`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type.

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `190601010600`, i.e. from 1st of January 1906 (way back!), and the timestamp for the latest observation is `201910012350`.

The information in this column is stored as integer values.

We now want to aggregate the data on a monthly level, and in order to do so we need to "label" each row of data based on the month when the record was observed. In order to do this, we need to somehow separate information about the year and month for each row.

We can create these "labels" by making a new column containing information about the month (including the year, but excluding day, hours, and minutes).

Before further taking that step, we should first convert the contents in the `TIME` column to a new column with character string values for convenience.

In [ ]:
# Convert to string


### String slicing

Now that we have converted the date and time information into character strings, we next need to "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year and month!

Based on this information, we can slice the correct range of characters from the `TIME_STR` column using the [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html) method.

In [ ]:
# Slice the string


# Let's see what we have
data.head()

Nice! Now we have "labeled" the rows based on information about date and time, but only including the year and month in the labels.

### Check your understanding

Create a new column called `'MONTH'` with information about the month of each observation, but excluding the year.

In [ ]:
# Enter your solution below
# Extract information about month from the TIME_STR column into a new column 'MONTH':


In [ ]:
# Check the result


### Datetime (*optional*)

In pandas, we can also convert dates and times into a new data type called [datetime](https://docs.python.org/3.7/library/datetime.html) using the [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function.

In [ ]:
# Convert character strings to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])

In [ ]:
# Check the output
data["DATE"].head()

#### Pandas Series datetime properties

There are several methods available for accessing information about the properties of datetime values. You can read more about datetime properties [from the pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).

With the datetime column, we can now extract different time units using the [pandas.Series.dt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) accessor.

In [ ]:
data["DATE"].dt.year

In [ ]:
data["DATE"].dt.month

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

In [ ]:
data["DATE"].dt.year.nunique()

For the final analysis, we need combined information of the year and month. One way to achieve this is to use the  `format` parameter to define the output datetime format according to [strftime(format)](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) method:

In [ ]:
# Convert to datetime and keep only year and month
data["YEAR_MONTH_DT"] = pd.to_datetime(data["TIME_STR"], format="%Y%m", exact=False)

**Note**: `exact=False` finds the characters matching the specified format and drops out the rest (days, hours and minutes are excluded in the output).

In [ ]:
data["YEAR_MONTH_DT"]

Now we have a unique label for each month as a datetime object!

## Aggregating data in pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), which is a handy method for combining large amounts of data and computing statistics for subgroups.

In our case, we will use the groupby method to calculate the average temperatures for each month through these three steps:

  1. Grouping the data based on the year and month
  2. Calculating the average for each month (each group) 
  3. Storing those values into a new DataFrame called `monthly_data`

Before we start grouping the data, let's once again see what our data looks like.

In [ ]:
print(f"number of rows: {len(data)}")

In [ ]:
data.head()

We have quite a few rows of weather data, and several observations per day. Our goal is to create an aggreated data frame that would have only one row per month.

To condense our data to monthly average values we can group our data based on the unique year and month combinations.

**Note**: It is also possible to create combinations of years and months on-the-fly when grouping the data:
    
```python
# Group the data 
grouped = data.groupby(['YEAR', 'MONTH'])
```

Now, let's explore the new variable `grouped`.

In [ ]:
type(grouped)

In [ ]:
len(grouped)

We have a new object with type `DataFrameGroupBy` with 601 groups. In order to understand what just happened, let's also check the number of unique year and month combinations in our data.

In [ ]:
data["YEAR_MONTH"].nunique()

Length of the grouped object should be the same as the number of unique values in the column we used for grouping. For each unique value, there is a group of data.

Let's explore our grouped data even further. 

We can check the "names" of each group.

In [ ]:
# Next line will print out all 601 group "keys"
# This is commented out here to avoid excess output to the course website
grouped.groups.keys()

### Accessing data for one group

Let us now check the contents for the group representing August 2019 (the name of that group is `201908`. We can get the values of that hour from the grouped object using the `get_group()` method.

In [ ]:
# Specify a month (as character string)


# Select the group


In [ ]:
# Let's see what we have


Ahaa! As we can see, a single group contains a DataFrame with values only for that specific month and year. Let's check the DataType of this group.

So, as noted above, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics, etc. for this specific group. We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` function that we already did during Lesson 5. Let's calculate the mean for following attributes all at once:

- `DIR`
- `SPEED`
- `GUST`
- `TEMP`
- `TEMP_C`
- `MONTH`

In [ ]:
# Specify the columns that will be part of the calculation
mean_cols = ["DIR", "SPEED", "GUST", "TEMP_F", "TEMP_C"]

# Calculate the mean values all at one go


# Let's see what we have


Above, we saw how you can access data from a single group. In order to get information about all groups (all months) we can use a `for` loop or methods available in the grouped object.

### For loops and grouped objects

When iterating over the groups in our `DataFrameGroupBy` object it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

So, let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening for the first group).

In [ ]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print(f"Key:\n {key}")
    print(f"\nFirst rows of data in this group:\n {group.head()}")

    # Stop iteration with break command
    break

OK, so from here we can see that the `key` contains the name of the group `YEARMO`.

Let's build on this and see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. We will repeat some of the earlier steps here so you can see and better understand what is happening.

In [ ]:
# Create an empty DataFrame for the aggregated values


# The columns that we want to aggregate
mean_cols = ["DIR", "SPEED", "GUST", "TEMP_F", "TEMP_C"]

# Iterate over the groups


    # Calculate mean
    

    # Add the ´key´ (i.e. the date+time information) into the aggregated values
    
    
    # Convert the mean_values series to a DataFrame and make it have a row orientation
    

    # Concatenate the aggregated values into the monthly_data DataFrame
    

**Note**: You can find a slightly more efficient approach for this same kind of aggregation in [Chapter 3 of the *Python for Geographic Data Analysis*](https://python-gis-book.readthedocs.io/en/latest/part1/chapter-03/nb/02-data-analysis.html#aggregating-data-with-groupby) textbook.

Now, let us see what we have.

In [ ]:
print(monthly_data)

Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

### Finding the mean for all groups at once

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object.

## Detecting warm months

Now that we have aggregated our data on monthly level, all we need to do is to sort our results in order to check which years had the warmest April temperatures. A simple approach is to select all Aprils from the data, group the data and check which group(s) have the highest mean value.

We can start this by selecting all records that are from April (regardless of the year).

Next, we can take a subset of columns that might contain interesting information.

In [ ]:
aprils = aprils[["STATION_NUMBER", "TEMP_F", "TEMP_C", "YEAR_MONTH"]]

We can group by year and month.

And then we can calculate the mean for each group.

Finally, we can sort and check the highest temperature values. We can sort the data frame in a descending order to do this.

So, how did April 2019 rank at the Tampere Pirkkala observation station? 

## Repeating the data analysis with a larger dataset

To wrap up today's lesson, let's repeat the data analysis steps above for all the available data we have (!). First, it would be good to confirm the path to the directory where all the input data are located.

The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info, all in one long code cell.

In [ ]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(
    fp,
    delim_whitespace=True,
    usecols=["USAF", "YR--MODAHRMN", "DIR", "SPD", "GUS", "TEMP", "MAX", "MIN"],
    na_values=["*", "**", "***", "****", "*****", "******"],
)

# Rename the columns
new_names = {
    "USAF": "STATION_NUMBER",
    "YR--MODAHRMN": "TIME",
    "SPD": "SPEED",
    "GUS": "GUST",
    "TEMP": "TEMP_F",
}
data = data.rename(columns=new_names)

# Print info about the current input file:
print(f"STATION NUMBER: {data.at[0, 'STATION_NUMBER']}")
print(f"NUMBER OF OBSERVATIONS: {len(data)}")

# Create column
col_name = "TEMP_C"
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

# Convert TIME to string
data["TIME_STR"] = data["TIME"].astype(str)

# Parse year and month
data["MONTH"] = data["TIME_STR"].str.slice(start=5, stop=6).astype(int)
data["YEAR"] = data["TIME_STR"].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April
aprils = data[data["MONTH"] == 4]

# Take a subset of columns
aprils = aprils[["STATION_NUMBER", "TEMP_F", "TEMP_C", "YEAR", "MONTH"]]

# Group by year and month
grouped = aprils.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

In [ ]:
print(data.at[0, "STATION_NUMBER"])

At this point we will use the `glob()` function from the module `glob` to list our input files. glob is a handy function for finding files in a directrory that match a given pattern, for example.

In [ ]:
import glob

**Note**: We're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!

**Note**: If you are using Jupyter Lab installed on your own computer, then the `file_list` variable you should use is defined below:

```python
file_list = glob.glob(r"data/0*txt")
```

In [ ]:
print(f"Number of files in the list: {len(file_list)}")
print(file_list)

Now, you should have all the relevant file names in a list, and we can loop over the list using a for loop.

In [ ]:
for fp in file_list:
    print(fp)

In [ ]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(
        fp,
        delim_whitespace=True,
        usecols=["USAF", "YR--MODAHRMN", "DIR", "SPD", "GUS", "TEMP", "MAX", "MIN"],
        na_values=["*", "**", "***", "****", "*****", "******"],
    )

    # Rename the columns
    new_names = {
        "USAF": "STATION_NUMBER",
        "YR--MODAHRMN": "TIME",
        "SPD": "SPEED",
        "GUS": "GUST",
        "TEMP": "TEMP_F",
    }
    data = data.rename(columns=new_names)

    # Print info about the current input file:
    print(f"STATION NUMBER: {data.at[0, 'STATION_NUMBER']}")
    print(f"NUMBER OF OBSERVATIONS: {len(data)}")

    # Create column
    col_name = "TEMP_C"
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

    # Convert TIME to string
    data["TIME_STR"] = data["TIME"].astype(str)

    # Parse year and month
    data["MONTH"] = data["TIME_STR"].str.slice(start=5, stop=6).astype(int)
    data["YEAR"] = data["TIME_STR"].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April
    aprils = data[data["MONTH"] == 4]

    # Take a subset of columns
    aprils = aprils[["STATION_NUMBER", "TEMP_F", "TEMP_C", "YEAR", "MONTH"]]

    # Group by year and month
    grouped = aprils.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

So, what can we conclude about how warm April 2019 was in Finland? Was it actually the warmest April on record? If so, in which stations?